In [21]:
# @title Imports

import dataclasses
import datetime
import functools
import math
import re
from typing import Optional

import cartopy.crs as ccrs
#from google.cloud import storage
from graphcast import autoregressive
from graphcast import casting
from graphcast import checkpoint
from graphcast import data_utils
from graphcast import graphcast
from graphcast import normalization
from graphcast import rollout
from graphcast import xarray_jax
from graphcast import xarray_tree
from IPython.display import HTML
import ipywidgets as widgets
import haiku as hk
import jax
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import animation
import numpy as np
import xarray




def parse_file_parts(file_name):
  return dict(part.split("-", 1) for part in file_name.split("_"))


In [22]:
# @title Choose the model
# Rewrite by S.F. Sune, https://github.com/sfsun67.
'''
    We have three options. Acquiring from https://console.cloud.google.com/storage/browser/dm_graphcast:
    GraphCast - ERA5 1979-2017 - resolution 0.25 - pressure levels 37 - mesh 2to6 - precipitation input and output.npz
    GraphCast_operational - ERA5-HRES 1979-2021 - resolution 0.25 - pressure levels 13 - mesh 2to6 - precipitation output only.npz
    GraphCast_small - ERA5 1979-2015 - resolution 1.0 - pressure levels 13 - mesh 2to5 - precipitation input and output.npz
'''
# find the result in this path /root/data/params, and list of names of all files in the "params/", with the "params/" perfix removed from the names.
import os
import glob

# Define the directory path
dir_path_params = "/root/data/params"

# Use glob to get all file paths in the directory
file_paths_params = glob.glob(os.path.join(dir_path_params, "*"))

# Remove the directory path and the ".../params/" prefix from each file name
params_file_options = [os.path.basename(path) for path in file_paths_params]


random_mesh_size = widgets.IntSlider(
    value=4, min=4, max=6, description="Mesh size:")
random_gnn_msg_steps = widgets.IntSlider(
    value=4, min=1, max=32, description="GNN message steps:")
random_latent_size = widgets.Dropdown(
    options=[int(2**i) for i in range(4, 10)], value=32,description="Latent size:")
random_levels = widgets.Dropdown(
    options=[13, 37], value=13, description="Pressure levels:")


params_file = widgets.Dropdown(
    options=params_file_options,
    description="Params file:",
    layout={"width": "max-content"})

source_tab = widgets.Tab([
    widgets.VBox([
        random_mesh_size,
        random_gnn_msg_steps,
        random_latent_size,
        random_levels,
    ]),
    params_file,
])
source_tab.set_title(0, "Random")
source_tab.set_title(1, "Checkpoint")
widgets.VBox([
    source_tab,
    widgets.Label(value="Run the next cell to load the model. Rerunning this cell clears your selection.")
])


In [23]:
# @title Load the model

source = source_tab.get_title(source_tab.selected_index)

if source == "Random":
  params = None  # Filled in below
  state = {}
  model_config = graphcast.ModelConfig(
      resolution=0,
      mesh_size=random_mesh_size.value,
      latent_size=random_latent_size.value,
      gnn_msg_steps=random_gnn_msg_steps.value,
      hidden_layers=1,
      radius_query_fraction_edge_length=0.6)
  task_config = graphcast.TaskConfig(
      input_variables=graphcast.TASK.input_variables,
      target_variables=graphcast.TASK.target_variables,
      forcing_variables=graphcast.TASK.forcing_variables,
      pressure_levels=graphcast.PRESSURE_LEVELS[random_levels.value],
      input_duration=graphcast.TASK.input_duration,
  )
else:
  assert source == "Checkpoint"
  '''with gcs_bucket.blob(f"params/{params_file.value}").open("rb") as f:
    ckpt = checkpoint.load(f, graphcast.CheckPoint)'''
  
  with open(f"{dir_path_params}/{params_file.value}", "rb") as f:
    ckpt = checkpoint.load(f, graphcast.CheckPoint)
  
  params = ckpt.params
  state = {}

  model_config = ckpt.model_config
  task_config = ckpt.task_config
  print("Model description:\n", ckpt.description, "\n")
  print("Model license:\n", ckpt.license, "\n")

model_config

ModelConfig(resolution=0, mesh_size=4, latent_size=32, gnn_msg_steps=4, hidden_layers=1, radius_query_fraction_edge_length=0.6, mesh2grid_edge_normalization_factor=None)

In [24]:
# @title Get and filter the list of available example datasets
# Rewrite by S.F. Sune, https://github.com/sfsun67.
# find the result in this path /root/data/dataset, and list of names of all files in the "dataset/", with the "dataset/" perfix removed from the names.

# Define the directory path
dir_path_dataset = "/root/data/dataset"

# Use glob to get all file paths in the directory
file_paths_dataset = glob.glob(os.path.join(dir_path_dataset, "*"))

# Remove the directory path and the ".../params/" prefix from each file name
dataset_file_options = [os.path.basename(path) for path in file_paths_dataset]
#print("dataset_file_options: ", dataset_file_options)

# Remove "dataset-" prefix from each file name
dataset_file_options = [name.removeprefix("dataset-") for name in dataset_file_options]


def data_valid_for_model(
    file_name: str, model_config: graphcast.ModelConfig, task_config: graphcast.TaskConfig):
  file_parts = parse_file_parts(file_name.removesuffix(".nc"))
  #print("file_parts: ", file_parts)
  return (
      model_config.resolution in (0, float(file_parts["res"])) and
      len(task_config.pressure_levels) == int(file_parts["levels"]) and
      (
          ("total_precipitation_6hr" in task_config.input_variables and
           file_parts["source"] in ("era5", "fake")) or
          ("total_precipitation_6hr" not in task_config.input_variables and
           file_parts["source"] in ("hres", "fake"))
      )
  )


dataset_file = widgets.Dropdown(
    options=[
        (", ".join([f"{k}: {v}" for k, v in parse_file_parts(option.removesuffix(".nc")).items()]), option)
        for option in dataset_file_options
        if data_valid_for_model(option, model_config, task_config)
    ],
    description="Dataset file:",
    layout={"width": "max-content"})
widgets.VBox([
    dataset_file,
    widgets.Label(value="Run the next cell to load the dataset. Rerunning this cell clears your selection and refilters the datasets that match your model.")
])

In [25]:
# @title Load weather data

if not data_valid_for_model(dataset_file.value, model_config, task_config):
  raise ValueError(
      "Invalid dataset file, rerun the cell above and choose a valid dataset file.")

'''with gcs_bucket.blob(f"dataset/{dataset_file.value}").open("rb") as f:
  example_batch = xarray.load_dataset(f).compute()'''

with open(f"{dir_path_dataset}/dataset-{dataset_file.value}", "rb") as f:
  example_batch = xarray.load_dataset(f).compute()

assert example_batch.dims["time"] >= 3  # 2 for input, >=1 for targets

print(", ".join([f"{k}: {v}" for k, v in parse_file_parts(dataset_file.value.removesuffix(".nc")).items()]))

example_batch

source: era5, date: 2022-01-01, res: 1.0, levels: 13, steps: 40


<xarray.Dataset>
Dimensions:                       (lon: 360, lat: 181, time: 42, level: 13,
                                   batch: 1)
Coordinates:
  * lon                           (lon) float32 0.0 1.0 2.0 ... 358.0 359.0
  * lat                           (lat) float32 -90.0 -89.0 -88.0 ... 89.0 90.0
  * time                          (time) timedelta64[ns] 0 days 00:00:00 ... ...
  * level                         (level) int32 50 100 150 200 ... 850 925 1000
    datetime                      (batch, time) datetime64[ns] 2022-01-01 ......
Dimensions without coordinates: batch
Data variables: (12/14)
    geopotential_at_surface       (lat, lon) float32 2.735e+04 ... -0.07617
    land_sea_mask                 (lat, lon) float32 1.0 1.0 1.0 ... 0.0 0.0 0.0
    2m_temperature                (batch, time, lat, lon) float32 250.7 ... 2...
    mean_sea_level_pressure       (batch, time, lat, lon) float32 9.931e+04 ....
    10m_v_component_of_wind       (batch, time, lat, lon) float32 -0.4393 ......
    10m_u_component_of_wind       (batch, time, lat, lon) float32 1.309 ... -...
    ...                            ...
    temperature                   (batch, time, level, lat, lon) float32 238....
    geopotential                  (batch, time, level, lat, lon) float32 1.98...
    u_component_of_wind           (batch, time, level, lat, lon) float32 -0.0...
    v_component_of_wind           (batch, time, level, lat, lon) float32 0.00...
    vertical_velocity             (batch, time, level, lat, lon) float32 -0.0...
    specific_humidity             (batch, time, level, lat, lon) float32 2.89...

In [6]:
print("example_batch", example_batch)

example_batch <xarray.Dataset>
Dimensions:                       (lon: 360, lat: 181, level: 13, time: 3,
                                   batch: 1)
Coordinates:
  * lon                           (lon) float32 0.0 1.0 2.0 ... 358.0 359.0
  * lat                           (lat) float32 -90.0 -89.0 -88.0 ... 89.0 90.0
  * level                         (level) int32 50 100 150 200 ... 850 925 1000
  * time                          (time) timedelta64[ns] 00:00:00 ... 12:00:00
    datetime                      (batch, time) datetime64[ns] 2022-01-01 ......
Dimensions without coordinates: batch
Data variables: (12/14)
    geopotential_at_surface       (lat, lon) float32 2.735e+04 ... -0.07617
    land_sea_mask                 (lat, lon) float32 1.0 1.0 1.0 ... 0.0 0.0 0.0
    2m_temperature                (batch, time, lat, lon) float32 250.7 ... 2...
    mean_sea_level_pressure       (batch, time, lat, lon) float32 9.931e+04 ....
    10m_v_component_of_wind       (batch, time, lat, lon) f

In [26]:
# @title Choose training and eval data to extract
train_steps = widgets.IntSlider(
    value=1, min=1, max=example_batch.sizes["time"]-2, description="Train steps")
eval_steps = widgets.IntSlider(
    value=example_batch.sizes["time"]-2, min=1, max=example_batch.sizes["time"]-2, description="Eval steps")

widgets.VBox([
    train_steps,
    eval_steps,
    widgets.Label(value="Run the next cell to extract the data. Rerunning this cell clears your selection.")
])

In [27]:
# @title Extract training and eval data

train_inputs, train_targets, train_forcings = data_utils.extract_inputs_targets_forcings(
    example_batch, target_lead_times=slice("6h", f"{train_steps.value*6}h"),
    **dataclasses.asdict(task_config))

eval_inputs, eval_targets, eval_forcings = data_utils.extract_inputs_targets_forcings(
    example_batch, target_lead_times=slice("6h", f"{eval_steps.value*6}h"),
    **dataclasses.asdict(task_config))

print("All Examples:  ", example_batch.dims.mapping)
print("Train Inputs:  ", train_inputs.dims.mapping)
print("Train Targets: ", train_targets.dims.mapping)
print("Train Forcings:", train_forcings.dims.mapping)
print("Eval Inputs:   ", eval_inputs.dims.mapping)
print("Eval Targets:  ", eval_targets.dims.mapping)
print("Eval Forcings: ", eval_forcings.dims.mapping)


All Examples:   {'lon': 360, 'lat': 181, 'time': 42, 'level': 13, 'batch': 1}
Train Inputs:   {'batch': 1, 'time': 2, 'lat': 181, 'lon': 360, 'level': 13}
Train Targets:  {'batch': 1, 'time': 2, 'lat': 181, 'lon': 360, 'level': 13}
Train Forcings: {'batch': 1, 'time': 2, 'lat': 181, 'lon': 360}
Eval Inputs:    {'batch': 1, 'time': 2, 'lat': 181, 'lon': 360, 'level': 13}
Eval Targets:   {'batch': 1, 'time': 40, 'lat': 181, 'lon': 360, 'level': 13}
Eval Forcings:  {'batch': 1, 'time': 40, 'lat': 181, 'lon': 360}


In [9]:
train_inputs

<xarray.Dataset>
Dimensions:                       (batch: 1, time: 2, lat: 181, lon: 360,
                                   level: 13)
Coordinates:
  * lon                           (lon) float32 0.0 1.0 2.0 ... 358.0 359.0
  * lat                           (lat) float32 -90.0 -89.0 -88.0 ... 89.0 90.0
  * level                         (level) int32 50 100 150 200 ... 850 925 1000
  * time                          (time) timedelta64[ns] -1 days +18:00:00 00...
Dimensions without coordinates: batch
Data variables: (12/18)
    2m_temperature                (batch, time, lat, lon) float32 250.7 ... 2...
    mean_sea_level_pressure       (batch, time, lat, lon) float32 9.931e+04 ....
    10m_v_component_of_wind       (batch, time, lat, lon) float32 -0.4393 ......
    10m_u_component_of_wind       (batch, time, lat, lon) float32 1.309 ... -...
    total_precipitation_6hr       (batch, time, lat, lon) float32 0.0004331 ....
    temperature                   (batch, time, level, lat, lon) float32 238....
    ...                            ...
    year_progress_sin             (batch, time) float32 0.006986 0.01129
    year_progress_cos             (batch, time) float32 1.0 0.9999
    day_progress_sin              (batch, time, lon) float32 0.0 ... 0.9998
    day_progress_cos              (batch, time, lon) float32 1.0 ... 0.01745
    geopotential_at_surface       (lat, lon) float32 2.735e+04 ... -0.07617
    land_sea_mask                 (lat, lon) float32 1.0 1.0 1.0 ... 0.0 0.0 0.0

In [10]:
train_targets

<xarray.Dataset>
Dimensions:                  (batch: 1, time: 1, lat: 181, lon: 360, level: 13)
Coordinates:
  * lon                      (lon) float32 0.0 1.0 2.0 3.0 ... 357.0 358.0 359.0
  * lat                      (lat) float32 -90.0 -89.0 -88.0 ... 88.0 89.0 90.0
  * level                    (level) int32 50 100 150 200 ... 700 850 925 1000
  * time                     (time) timedelta64[ns] 06:00:00
Dimensions without coordinates: batch
Data variables:
    2m_temperature           (batch, time, lat, lon) float32 248.3 ... 247.4
    mean_sea_level_pressure  (batch, time, lat, lon) float32 9.983e+04 ... 1....
    10m_v_component_of_wind  (batch, time, lat, lon) float32 -0.04184 ... -0....
    10m_u_component_of_wind  (batch, time, lat, lon) float32 0.04857 ... -0.2023
    total_precipitation_6hr  (batch, time, lat, lon) float32 7.443e-05 ... 5....
    temperature              (batch, time, level, lat, lon) float32 238.1 ......
    geopotential             (batch, time, level, lat, lon) float32 1.983e+05...
    u_component_of_wind      (batch, time, level, lat, lon) float32 0.0006237...
    v_component_of_wind      (batch, time, level, lat, lon) float32 0.0003135...
    vertical_velocity        (batch, time, level, lat, lon) float32 0.001688 ...
    specific_humidity        (batch, time, level, lat, lon) float32 2.898e-06...

In [28]:
# @title Load normalization data
# Rewrite by S.F. Sune, https://github.com/sfsun67.
dir_path_stats = "/root/data/stats"

with open(f"{dir_path_stats}/stats-diffs_stddev_by_level.nc", "rb") as f:
  diffs_stddev_by_level = xarray.load_dataset(f).compute()
with open(f"{dir_path_stats}/stats-mean_by_level.nc", "rb") as f:
  mean_by_level = xarray.load_dataset(f).compute()
with open(f"{dir_path_stats}/stats-stddev_by_level.nc", "rb") as f:
  stddev_by_level = xarray.load_dataset(f).compute()

In [29]:
# @title Build jitted functions, and possibly initialize random weights
# Construct the model and initialize the weights.
# 构建模型并初始化权重

# 模型组网
# Construct the model
def construct_wrapped_graphcast(
    model_config: graphcast.ModelConfig,
    task_config: graphcast.TaskConfig):
  """Constructs and wraps the GraphCast Predictor."""
  # Deeper one-step predictor.
  predictor = graphcast.GraphCast(model_config, task_config)

  # Modify inputs/outputs to `graphcast.GraphCast` to handle conversion to
  # from/to float32 to/from BFloat16.
  predictor = casting.Bfloat16Cast(predictor)

  # Modify inputs/outputs to `casting.Bfloat16Cast` so the casting to/from
  # BFloat16 happens after applying normalization to the inputs/targets.
  predictor = normalization.InputsAndResiduals(
      predictor,
      diffs_stddev_by_level=diffs_stddev_by_level,
      mean_by_level=mean_by_level,
      stddev_by_level=stddev_by_level)

  # Wraps everything so the one-step model can produce trajectories.
  predictor = autoregressive.Predictor(predictor, gradient_checkpointing=True)
  return predictor

# 前向运算
# forward
@hk.transform_with_state
def run_forward(model_config, task_config, inputs, targets_template, forcings):
  predictor = construct_wrapped_graphcast(model_config, task_config)
  return predictor(inputs, targets_template=targets_template, forcings=forcings)

# 计算损失函数
# loss function
@hk.transform_with_state    # used to convert a pure function into a stateful function
def loss_fn(model_config, task_config, inputs, targets, forcings):
  predictor = construct_wrapped_graphcast(model_config, task_config)    # constructs and wraps a GraphCast Predictor, which is a model used for making predictions in a graph-based machine learning task.
  loss, diagnostics = predictor.loss(inputs, targets, forcings)
  return xarray_tree.map_structure(
      lambda x: xarray_jax.unwrap_data(x.mean(), require_jax=True),
      (loss, diagnostics))

# 计算梯度
# gradient
def grads_fn(params, state, model_config, task_config, inputs, targets, forcings):
  def _aux(params, state, i, t, f):
    (loss, diagnostics), next_state = loss_fn.apply(
        params, state, jax.random.PRNGKey(0), model_config, task_config,
        i, t, f)
    return loss, (diagnostics, next_state)
  (loss, (diagnostics, next_state)), grads = jax.value_and_grad(
      _aux, has_aux=True)(params, state, inputs, targets, forcings)
  return loss, diagnostics, next_state, grads

# Jax doesn't seem to like passing configs as args through the jit. Passing it
# in via partial (instead of capture by closure) forces jax to invalidate the
# jit cache if you change configs.
def with_configs(fn):
  return functools.partial(
      fn, model_config=model_config, task_config=task_config)

# Always pass params and state, so the usage below are simpler
def with_params(fn):
  return functools.partial(fn, params=params, state=state)

# Our models aren't stateful, so the state is always empty, so just return the
# predictions. This is requiredy by our rollout code, and generally simpler.
def drop_state(fn):
  return lambda **kw: fn(**kw)[0]

init_jitted = jax.jit(with_configs(run_forward.init))

if params is None:
  params, state = init_jitted(
      rng=jax.random.PRNGKey(0),
      inputs=train_inputs,
      targets_template=train_targets,
      forcings=train_forcings)

loss_fn_jitted = drop_state(with_params(jax.jit(with_configs(loss_fn.apply))))
grads_fn_jitted = with_params(jax.jit(with_configs(grads_fn)))
run_forward_jitted = drop_state(with_params(jax.jit(with_configs(
    run_forward.apply))))

# 测试：1. 对数据集进行迭代    2. 权重保存与加载

In [19]:
# 对数据集进行迭代   模型为原始的。数据为步长40的数据集, train step = 2
for i in range(39):
    example_batch_slice = example_batch.isel(time=slice(i, 4+i))
    train_inputs, train_targets, train_forcings = data_utils.extract_inputs_targets_forcings(
        example_batch_slice, target_lead_times=slice("6h", f"{train_steps.value*6}h"),
        **dataclasses.asdict(task_config))
    # @title Gradient computation (backprop through time)
    loss, diagnostics, next_state, grads = grads_fn_jitted(
        inputs=train_inputs,
        targets=train_targets,
        forcings=train_forcings)
    mean_grad = np.mean(jax.tree_util.tree_flatten(jax.tree_util.tree_map(lambda x: np.abs(x).mean(), grads))[0])
    print(f"Loss: {loss:.4f}, Mean |grad|: {mean_grad:.6f}")



Loss: 26.0938, Mean |grad|: 0.251308
Loss: 24.3750, Mean |grad|: 0.217100
Loss: 24.5000, Mean |grad|: 0.224518
Loss: 25.7188, Mean |grad|: 0.267907
Loss: 25.7500, Mean |grad|: 0.269001
Loss: 23.8125, Mean |grad|: 0.217672
Loss: 23.8125, Mean |grad|: 0.221061
Loss: 24.7812, Mean |grad|: 0.255847
Loss: 25.2812, Mean |grad|: 0.254832
Loss: 24.4688, Mean |grad|: 0.218144
Loss: 24.5625, Mean |grad|: 0.218410
Loss: 25.9375, Mean |grad|: 0.256558
Loss: 26.9062, Mean |grad|: 0.259060
Loss: 25.7812, Mean |grad|: 0.223054
Loss: 26.0000, Mean |grad|: 0.224861
Loss: 27.4375, Mean |grad|: 0.260466
Loss: 28.0000, Mean |grad|: 0.260654
Loss: 26.1875, Mean |grad|: 0.219026
Loss: 25.8125, Mean |grad|: 0.222837
Loss: 26.5312, Mean |grad|: 0.263233
Loss: 26.6875, Mean |grad|: 0.259255
Loss: 24.8438, Mean |grad|: 0.220437
Loss: 25.0312, Mean |grad|: 0.226548
Loss: 26.1562, Mean |grad|: 0.265268
Loss: 26.8438, Mean |grad|: 0.260144
Loss: 25.5938, Mean |grad|: 0.221645
Loss: 25.7188, Mean |grad|: 0.227682
L

In [30]:
# 对数据集进行迭代  0-20
for i in range(20):
    example_batch_slice = example_batch.isel(time=slice(i, 4+i))
    train_inputs, train_targets, train_forcings = data_utils.extract_inputs_targets_forcings(
        example_batch_slice, target_lead_times=slice("6h", f"{train_steps.value*6}h"),
        **dataclasses.asdict(task_config))
    # @title Gradient computation (backprop through time)
    loss, diagnostics, next_state, grads = grads_fn_jitted(
        inputs=train_inputs,
        targets=train_targets,
        forcings=train_forcings)
    mean_grad = np.mean(jax.tree_util.tree_flatten(jax.tree_util.tree_map(lambda x: np.abs(x).mean(), grads))[0])
    print(f"Loss: {loss:.4f}, Mean |grad|: {mean_grad:.6f}")

Loss: 26.0938, Mean |grad|: 0.251319
Loss: 24.3750, Mean |grad|: 0.217114
Loss: 24.5000, Mean |grad|: 0.224506
Loss: 25.7188, Mean |grad|: 0.267946
Loss: 25.7500, Mean |grad|: 0.269030
Loss: 23.8125, Mean |grad|: 0.217632
Loss: 23.8125, Mean |grad|: 0.221011
Loss: 24.7812, Mean |grad|: 0.255858
Loss: 25.2812, Mean |grad|: 0.254751
Loss: 24.4688, Mean |grad|: 0.218168
Loss: 24.5625, Mean |grad|: 0.218437
Loss: 25.9375, Mean |grad|: 0.256492
Loss: 26.9062, Mean |grad|: 0.259096
Loss: 25.7812, Mean |grad|: 0.223055
Loss: 26.0000, Mean |grad|: 0.224869
Loss: 27.4375, Mean |grad|: 0.260419
Loss: 28.0000, Mean |grad|: 0.260706
Loss: 26.1875, Mean |grad|: 0.219043
Loss: 25.8125, Mean |grad|: 0.222753
Loss: 26.5312, Mean |grad|: 0.263238


In [33]:
print(ckpt)

CheckPoint(params={'grid2mesh_gnn/~_networks_builder/encoder_edges_grid2mesh_layer_norm': {'offset': array([-3.09514757e-02, -4.78553193e-06,  2.64992355e-04,  2.07589462e-01,
       -1.29042447e-01,  6.21634535e-02,  9.64820065e-05,  2.58372426e-01,
       -2.32808889e-04, -9.75188985e-02, -4.45864316e-05, -8.39245622e-05,
       -1.01761361e-05,  1.91704243e-01,  7.94045478e-02,  3.76497179e-01,
       -5.23938943e-05,  1.61429390e-01,  5.35177751e-05,  1.01985842e-01,
        5.95859662e-02, -3.10882568e-01, -2.21688762e-01, -4.78984330e-05,
        3.79683614e-01,  3.57430458e-01, -2.07171150e-04, -2.52931118e-01,
       -6.27476349e-02,  3.88838500e-01, -2.30848741e-06, -1.51016039e-03,
       -5.03354035e-02, -3.85118455e-01,  3.04043800e-01,  1.81791261e-01,
       -9.16860154e-06,  5.45367748e-02, -1.29305691e-01, -9.14223929e-05,
        2.05694035e-01,  2.99708664e-01, -5.90807285e-05,  1.40228152e-01,
        2.42122352e-01, -2.71377563e-01, -2.38817543e-01, -1.48481533e-01,

In [13]:
# 存储模型
#checkpoint.dump(ckpt)  # 保存模型权重
checkpoint.dump(model_config)  # 保存模型描述
checkpoint.dump(task_config)  # 保存任务描述


TypeError: dump() missing 1 required positional argument: 'value'

In [ ]:
# 加载训练20次的模型

with open(f"{dir_path_params}/{params_file.value}", "rb") as f:
    ckpt = checkpoint.load(f, graphcast.CheckPoint)

params = ckpt.params
state = {}

model_config = ckpt.model_config
task_config = ckpt.task_config
print("Model description:\n", ckpt.description, "\n")
print("Model license:\n", ckpt.license, "\n")

In [15]:
#继续迭代
for i in range(20,39):
    example_batch_slice = example_batch.isel(time=slice(i, 4+i))
    train_inputs, train_targets, train_forcings = data_utils.extract_inputs_targets_forcings(
        example_batch_slice, target_lead_times=slice("6h", f"{train_steps.value*6}h"),
        **dataclasses.asdict(task_config))
    # @title Gradient computation (backprop through time)
    loss, diagnostics, next_state, grads = grads_fn_jitted(
        inputs=train_inputs,
        targets=train_targets,
        forcings=train_forcings)
    mean_grad = np.mean(jax.tree_util.tree_flatten(jax.tree_util.tree_map(lambda x: np.abs(x).mean(), grads))[0])
    print(f"Loss: {loss:.4f}, Mean |grad|: {mean_grad:.6f}")

KeyboardInterrupt: 

# Run the model

Note that the cell below may take a while (possibly minutes) to run the first time you execute them, because this will include the time it takes for the code to compile. The second time running will be significantly faster.

This use the python loop to iterate over prediction steps, where the 1-step prediction is jitted. This has lower memory requirements than the training steps below, and should enable making prediction with the small GraphCast model on 1 deg resolution data for 4 steps.

In [ ]:
# @title Autoregressive rollout (loop in python)

assert model_config.resolution in (0, 360. / eval_inputs.sizes["lon"]), (
  "Model resolution doesn't match the data resolution. You likely want to "
  "re-filter the dataset list, and download the correct data.")

print("Inputs:  ", eval_inputs.dims.mapping)
print("Targets: ", eval_targets.dims.mapping)
print("Forcings:", eval_forcings.dims.mapping)

predictions = rollout.chunked_prediction(
    run_forward_jitted,
    rng=jax.random.PRNGKey(0),
    inputs=eval_inputs,
    targets_template=eval_targets * np.nan,
    forcings=eval_forcings)
predictions

Inputs:   {'batch': 1, 'time': 2, 'lat': 181, 'lon': 360, 'level': 13}
Targets:  {'batch': 1, 'time': 1, 'lat': 181, 'lon': 360, 'level': 13}
Forcings: {'batch': 1, 'time': 1, 'lat': 181, 'lon': 360}


<xarray.Dataset>
Dimensions:                  (time: 1, batch: 1, lat: 181, lon: 360, level: 13)
Coordinates:
  * time                     (time) timedelta64[ns] 06:00:00
  * lon                      (lon) float32 0.0 1.0 2.0 3.0 ... 357.0 358.0 359.0
  * lat                      (lat) float32 -90.0 -89.0 -88.0 ... 88.0 89.0 90.0
  * level                    (level) int32 50 100 150 200 ... 700 850 925 1000
Dimensions without coordinates: batch
Data variables:
    10m_u_component_of_wind  (time, batch, lat, lon) float32 3.401 ... -0.997
    10m_v_component_of_wind  (time, batch, lat, lon) float32 -3.368 ... -0.4125
    2m_temperature           (time, batch, lat, lon) float32 250.2 ... 246.9
    geopotential             (time, batch, level, lat, lon) float32 1.985e+05...
    mean_sea_level_pressure  (time, batch, lat, lon) float32 9.978e+04 ... 1....
    specific_humidity        (time, batch, level, lat, lon) float32 2.91e-06 ...
    temperature              (time, batch, level, lat, lon) float32 237.9 ......
    total_precipitation_6hr  (time, batch, lat, lon) float32 -0.003617 ... -0...
    u_component_of_wind      (time, batch, level, lat, lon) float32 1.323 ......
    v_component_of_wind      (time, batch, level, lat, lon) float32 -3.226 .....
    vertical_velocity        (time, batch, level, lat, lon) float32 -0.000568...

# Train the model

The following operations require a large amount of memory and, depending on the accelerator being used, will only fit the very small "random" model on low resolution data. It uses the number of training steps selected above.

The first time executing the cell takes more time, as it include the time to jit the function.

In [ ]:
# @title Loss computation (autoregressive loss over multiple steps)
loss, diagnostics = loss_fn_jitted(
    rng=jax.random.PRNGKey(0),
    inputs=train_inputs,
    targets=train_targets,
    forcings=train_forcings)

print("Loss:", float(loss))

ValueError: 'grid2mesh_gnn/~_networks_builder/encoder_nodes_grid_nodes_mlp/~/linear_0/w' with retrieved shape (186, 32) does not match shape=[98, 32] dtype=dtype(bfloat16)

In [ ]:
# @title Gradient computation (backprop through time)
loss, diagnostics, next_state, grads = grads_fn_jitted(
    inputs=train_inputs,
    targets=train_targets,
    forcings=train_forcings)
mean_grad = np.mean(jax.tree_util.tree_flatten(jax.tree_util.tree_map(lambda x: np.abs(x).mean(), grads))[0])
print(f"Loss: {loss:.4f}, Mean |grad|: {mean_grad:.6f}")

In [ ]:
# @title Autoregressive rollout (keep the loop in JAX)
print("Inputs:  ", train_inputs.dims.mapping)
print("Targets: ", train_targets.dims.mapping)
print("Forcings:", train_forcings.dims.mapping)

predictions = run_forward_jitted(
    rng=jax.random.PRNGKey(0),
    inputs=train_inputs,
    targets_template=train_targets * np.nan,
    forcings=train_forcings)
predictions